# Segmenting and Clustering Neighborhoods in Toronto

first we import all the libraries we might need

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge geocoder --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


because i did not know how to get the table in the 'normal' way (as json?) I use Beautiful soup and print it

In [2]:
from bs4 import BeautifulSoup #import beautifulsoup

In [3]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #define url variable

In [4]:
soup = BeautifulSoup(wiki_url,'html.parser') #parse it
print(soup.prettify()) #print it

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptUTApAMMQAASM08X8AAABC","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

I just want to find the table-information 

In [5]:
table = soup.find('table',{'class':'wikitable sortable'}).tbody #just take the table body from the website
print(table) #have a closer look into the table

<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East York
</td>
<td>Parkview Hill / W

recognize all new rows start with 'tr', the columns with 'th' and new cells with 'td' we define variables for the rows and columns and the dataframe itself

In [60]:
rows = table.find_all('tr') #define the rows by finding all lines with tr
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')] #define the columns by going through the first headline and find all the lines with th (because the new line operators \n replace them by nothing)
print(columns) #print for checking

['Postal code', 'Borough', 'Neighborhood']


In [61]:
toronto_df = pd.DataFrame(columns=columns) #define the panda dataframe
print(toronto_df)

Empty DataFrame
Columns: [Postal code, Borough, Neighborhood]
Index: []


fill the dataframe with data

In [62]:
for i in range(1,len(rows)): #going through all rows (skip the headline)
    tds = rows[i].find_all('td') #define the all cells
    values = [td.text.replace('\n','').replace(' /',',') for td in tds]  #define the cell-values and replace the the unwanted elements   
    print(values)
    toronto_df = toronto_df.append(pd.Series(values, index = columns),ignore_index = True) #append it to the dataframe
      
print(toronto_df)

['M1A', 'Not assigned', '']
['M2A', 'Not assigned', '']
['M3A', 'North York', 'Parkwoods']
['M4A', 'North York', 'Victoria Village']
['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']
['M6A', 'North York', 'Lawrence Manor, Lawrence Heights']
['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"]
['M8A', 'Not assigned', '']
['M9A', 'Etobicoke', 'Islington Avenue']
['M1B', 'Scarborough', 'Malvern, Rouge']
['M2B', 'Not assigned', '']
['M3B', 'North York', 'Don Mills']
['M4B', 'East York', 'Parkview Hill, Woodbine Gardens']
['M5B', 'Downtown Toronto', 'Garden District, Ryerson']
['M6B', 'North York', 'Glencairn']
['M7B', 'Not assigned', '']
['M8B', 'Not assigned', '']
['M9B', 'Etobicoke', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale']
['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek']
['M2C', 'Not assigned', '']
['M3C', 'North York', 'Don Mills']
['M4C', 'East York', 'Woodbine Heights']
['M5C', 'Downtown Toronto', 'St. Jame

just keep the rows where the Borough is not 'Not assigned' and reset the index

In [63]:
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
toronto_df = toronto_df.reset_index(drop=True)
print(toronto_df)

    Postal code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
2

check the shape of the dataframe (end of the first part)

In [64]:
toronto_df.shape

(103, 3)

I tried to use the geocoder but after some tests I recognized that it always just returns None as value.
Anyway, if it would work I would use the following code (Do not run it, it would fill the rows with empty values). 
Do you had the same problems?

In [38]:
import geocoder

for row in toronto_df.iterrows():
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(toronto_df[postal_code]))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
        
    toronto_df = toronto_df.append({'Latitude': latitude,
                                    'Longitude': longitude}, ignore_index=True)



so I used the given csv

In [55]:
toronto_latlng_df = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [65]:
toronto_latlng_df.shape

(103, 3)

How you can see it matches the shape of the first dataframe but the order by the Postal Code is different.
So I thought propably the postal code columns would have different types.
Hence, I tried to cast the column as string typ and reorder the Dataframe but I had to recognized it does not change:

In [57]:
toronto_latlng_df['Postal Code'].astype(str)
toronto_latlng_df.sort_values(by=['Postal Code'], ascending=False)
toronto_latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then I checked the types of both columns in the dataframes (how you can see they are both from type object)

In [29]:
print(toronto_latlng_df.dtypes)

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object


In [32]:
print(toronto_df.dtypes)

Postal code     object
Borough         object
Neighborhood    object
dtype: object


Even so I tried the same with the first dataframe but I does not change as well. Recognize that the postal code columns are not written the same, so first I rename the Column. 

In [66]:
toronto_df.rename(columns={'Postal code': 'Postal Code'}, inplace=True) #rename the columnname
toronto_df['Postal Code'].astype(str)
toronto_df.sort_values(by='Postal Code', ascending=False)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Because of this I wanted to build a loop to compare the Postal Codes and append the coordinates if they match, but it does not work and I cannot find the error. Do you find it?

In [67]:
for row in toronto_df.iterrows():
    if toronto_df.loc[row,'Postal Code'] == toronto_latlng_df.loc[row,'Postal Code']:
        toronto_df = toronto_df.append({'Latitude':toronto_latlng_df.loc[row,'Latitude'],'Longitude': toronto_latlng_df.loc[row,'Longitude']}, ignore_index=True)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

Because I don't know to sort it right, I just joined them both to continue with the task even with wrong assigned values. To my surprise the coordinates were attached right to the Neighborhoods by the merge-method.

In [75]:
df_complete = pd.merge(toronto_df, toronto_latlng_df, on='Postal Code', how='outer')
df_complete

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [87]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


create the map,
go through the dataframe to add markers,
print the map

In [77]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_complete['Latitude'], df_complete['Longitude'], df_complete['Borough'], df_complete['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

select just the Boroughs who contain 'Toronto'

In [85]:
selected_toronto_df = df_complete[df_complete.Borough.str.contains('Toronto',case=False)]
selected_toronto_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


this should be enough for 12 points, so I skip the rest